In [ ]:
!pip install -q -U torch
!pip install -q -U peft datasets transformers trl einops

In [ ]:
import os
import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, AdaLoraModel, AdaLoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from tqdm import tqdm
from trl import SFTTrainer

In [ ]:
# Model
base_model = "microsoft/phi-2"
new_model = "phi-2-medquad"

# Dataset
dataset = load_dataset("prsdm/MedQuad-phi2-1k", split="train[:75]")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map={"": 0},
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train_dataset = dataset.select(range(50))
test_dataset = dataset.select(range(51,75))

In [ ]:
def find_target_modules(model):
    unique_layers = set()

    for name, module in model.named_modules():
        if "Linear" in str(type(module)):
            layer_type = name.split('.')[-1]
            unique_layers.add(layer_type)
    return list(unique_layers)

In [ ]:
find_target_modules(model)

['v_proj', 'fc2', 'fc1', 'k_proj', 'lm_head', 'dense', 'q_proj']

In [ ]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    report_to="none",
    num_train_epochs = 1,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "adamw_torch",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 50,
)

In [ ]:
peft_config = AdaLoraConfig(
  peft_type="ADALORA",
  # task_type="SEQ_2_SEQ_LM",
  r=8,
  lora_alpha=32,
  target_modules=['k_proj', 'fc1', 'fc2', 'q_proj', 'v_proj'],
  lora_dropout=0.01,
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
# Training the model and getting the training output
train_output = trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
50,0.965500


In [ ]:
train_output

TrainOutput(global_step=50, training_loss=0.9654789733886718, metrics={'train_runtime': 586.4646, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.085, 'total_flos': 1426771426935360.0, 'train_loss': 0.9654789733886718, 'epoch': 1.0})

In [ ]:
# inference

for prompt in test_dataset.select(range(1)):
  # Extract the instruction part
  instruction = prompt["text"].split('### Instruction: ')[1].split('### Assistant: ')[0]
  print(instruction)

How to diagnose Cushing's Syndrome ? 


In [ ]:
for prompt in test_dataset.select(range(1)):
  print(prompt["text"])

### Instruction: How to diagnose Cushing's Syndrome ? ### Assistant: Diagnosis is based on a review of a person's medical history, a physical examination, and laboratory tests. X rays of the adrenal or pituitary glands can be useful in locating tumors.
                
Tests to Diagnose Cushing's Syndrome
                
No single lab test is perfect and usually several are needed. The three most common tests used to diagnose Cushing's syndrome are the 24-hour urinary free cortisol test, measurement of midnight plasma cortisol or late-night salivary cortisol, and the low-dose dexamethasone suppression test. Another test, the dexamethasone-corticotropin-releasing hormone test, may be needed to distinguish Cushing's syndrome from other causes of excess cortisol.
                
- 24-hour urinary free cortisol level. In this test, a person's urine is collected several times over a 24-hour period and tested for cortisol. Levels higher than 50 to 100 micrograms a day for an adult suggest 

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)

# List to store results
results = []

# Iterating over the testing dataset and generating predictions
for prompt in test_dataset.select(range(1)):
    # Extract the instruction part
    instruction = prompt["text"].split('### Instruction: ')[1].split('### Assistant: ')[0]
    result = pipe(f"### Instruction: {instruction}")
    generated_text = result[0]['generated_text']
    results.append({
        "prompt": instruction,
        "generated_text": generated_text
    })

# Printing the required result
print(f"{results[-1]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


### Instruction: How to diagnose Cushing's Syndrome ? 

To diagnose Cushing's Syndrome, doctors may perform a variety of tests, including:

1. Blood tests: These tests can measure the levels of cortisol in the blood.

2. Urine tests: These tests can measure the levels of cortisol in the urine.

3. Imaging tests: These tests can show the size and shape of the adrenal glands.

4. Biopsy: This test involves taking a small sample of the adrenal gland tissue to examine under a microscope.

5. ACTH stimulation test: This test involves injecting a synthetic form of ACTH into the body and measuring the levels of cortisol in the blood.

6. MRI: This test uses magnetic fields and radio waves to create detailed images of the adrenal glands.

7. CT scan: This test uses X-rays and computer technology to create detailed images of the adrenal glands.

8. Dexamethasone suppression test: This test involves taking a synthetic form of cortisol and measuring the levels of cortisol in the blood.

9. 24-hou